# Agent Frameworks: LangChain Patterns with Local Models

This notebook explores agent architecture patterns using **LangChain** with local Ollama models. While there are multiple agent frameworks available (AWS Strands, CrewAI, AutoGen, LlamaIndex), we focus on LangChain because:

- **Mature ecosystem** with extensive documentation and community support
- **Local model compatibility** works seamlessly with Ollama
- **Flexible patterns** supports multiple agent architectures
- **Production ready** with robust error handling and monitoring
- **Consistency** with other notebooks in this collection

## Core Agent Concepts Covered:
1. **Agent Architecture Patterns** (ReAct, Plan-and-Execute, Multi-agent)
2. **Tool Integration and Function Calling**
3. **Memory Management and State Persistence** 
4. **Agent Orchestration and Workflow Management**

In [76]:
# Install required packages
# !pip install langchain langchain-ollama langchain-community
# Ensure Ollama models are available:
# ollama pull gemma3
# ollama pull nomic-embed-text

In [77]:
import os
from langchain_ollama import ChatOllama
from langchain.agents import create_react_agent, AgentExecutor
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun

# Setup Ollama LLM
llm = ChatOllama(model="gemma3", temperature=0)

## 1. Simple ReAct Agent

ReAct (Reasoning + Acting) pattern combines reasoning and action in a loop.

In [78]:
# Define tools
def calculator(expression: str) -> str:
    """Calculate mathematical expressions"""
    try:
        return str(eval(expression))
    except:
        return "Invalid expression"

search = DuckDuckGoSearchRun()

tools = [
    Tool(name="Calculator", func=calculator, description="Calculate math expressions"),
    Tool(name="Search", func=search.run, description="Search the internet")
]

# Create ReAct agent
react_prompt = PromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

agent = create_react_agent(llm, tools, react_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [79]:
# Test ReAct agent
question = "What is 25 * 4 and what's the latest news about AI?"
result = agent_executor.invoke({"input": question})
print(f"Result: {result['output']}")



> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Okay, let's break this down. First, I need to calculate 25 * 4. Then, I need to find the latest news about AI.
Action: Calculator
Action Input: "25 * 4"100

INFO:ddgs.ddgs:Error in engine wikipedia: DDGSException("RuntimeError: RuntimeError('error sending request for url (https://wt.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=latest%20news%20about%20AI): client error (Connect)\\n\\nCaused by:\\n    0: client error (Connect)\\n    1: dns error: failed to lookup address information: nodename nor servname provided, or not known\\n    2: failed to lookup address information: nodename nor servname provided, or not known')")


Okay, I have the result of the calculation. Now I need to find the latest news about AI.
Action: Search
Action Input: "latest news about AI"AI News reports on the latest artificial intelligence news and insights. Explore industry trends from the frontline of AI . It’s my favorite AI story as of late , probably because it captures so well the chaos that can unfold when people actually use AI the way tech companies have all but told them to. As the writer of the story, Laurie Clarke, points out, it’s not a total pipe dream that AI could be therapeutically useful. Snowflake (SNOW) also ticked up following rival Databricks' latest $100 billion private valuation and news that its AI products now exceed a $1 billion revenue run-rate. Meanwhile, Oracle advanced ahead of its earnings release, with Wall Street raising price targets in anticipation of... The Trump administration, in its recent AI Action Plan, seemed to signal that Grok’s offensiveness might not be a problem — it included an anti

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Final Answer: 25 * 4 = 100. The latest news about AI includes reports on Snowflake's stock performance, Databricks' valuation, the Trump administration's AI Action Plan, Grok's offensiveness, and xAI's contract with the DoD.

> Finished chain.
Result: 25 * 4 = 100. The latest news about AI includes reports on Snowflake's stock performance, Databricks' valuation, the Trump administration's AI Action Plan, Grok's offensiveness, and xAI's contract with the DoD.


## 2. Memory Management and State Persistence

In [80]:
# Add memory to agent with proper prompt template
from langchain.prompts import PromptTemplate

# Create memory-aware prompt template
memory_prompt = PromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Previous conversation:
{chat_history}

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

# Use string-based memory instead of message-based
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=False)

# Create agent with memory-aware prompt
memory_agent = create_react_agent(llm, tools, memory_prompt)
agent_with_memory = AgentExecutor(
    agent=memory_agent, 
    tools=tools, 
    memory=memory,
    verbose=True
)

# Test conversation memory
print("First interaction:")
result1 = agent_with_memory.invoke({"input": "My name is John"})
print(f"Response: {result1['output']}")

print("\nSecond interaction:")
result2 = agent_with_memory.invoke({"input": "What's my name?"})
print(f"Response: {result2['output']}")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


First interaction:


> Entering new AgentExecutor chain...


INFO:ddgs.ddgs:Error in engine wikipedia: DDGSException("RuntimeError: RuntimeError('error sending request for url (https://wt.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=My%20name%20is%20John): client error (Connect)\\n\\nCaused by:\\n    0: client error (Connect)\\n    1: dns error: failed to lookup address information: nodename nor servname provided, or not known\\n    2: failed to lookup address information: nodename nor servname provided, or not known')")


Action: Search
Action Input: "My name is John"Tracia Traill My name is John Taylor "the Bass God " True Wild Boys story. My dad kept telling me to turn the stereo down. Intro to Wild Boys started I ... __ ABOUT Mynameisjohn is John Lillis, an artist, DJ and record collector living in Liscannor, Co. Clare. John has over 20 years experience DJing around ... “My name is John Cena .” “The name I use in the ring is John Cena.” “My government name is John Cena.” John Cena was born to be a wrestler and says he'd do it ... The song was written while Duran Duran were on tour , the music was written by the band, and lyrics by Simon Le Bon. The story behind the lyrics was told by both ... To edit the info that you use on Google services, like your name and photo, sign in to your account. You can also choose what personal info to show when you interact with others on Google services.

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


...

I need to find out John's name. I can use the Search tool for this.
Action: Search
Action Input: "What's my name?"Featuring guest vocals from Canadian rapper Drake, the song was released as the second single from Loud on October 25, 2010, through Def Jam Recordings and in Germany on January 21, 2011, as a single through Universal Music Group. Kicking off in San Diego, CA on July 17, the arena tour will celebrate the chart-topping music from Disney’ s “Descendants” and “Zombies” movie franchises, inspiring families, and fans of all ages... Oct 26, 2010 · A song by Rihanna featuring Drake about their lovers and their names. The lyrics include romantic bars, sexual innuendos, and a catchy chorus of "oh na-na, what's my name?" Featuring guest vocals from Canadian rapper Drake, the song was released as the second single from Loud on October 25, 2010, through Def Jam Recordings and in Germany on January 21, 2011, as a single through Universal Music Group. Rihanna " What ' s My Name ?": Oh na

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:ddgs.ddgs:Error in engine wikipedia: DDGSException("RuntimeError: RuntimeError('error sending request for url (https://wt.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=What%27s%20my%20name%3F): client error (Connect)\\n\\nCaused by:\\n    0: client error (Connect)\\n    1: dns error: failed to lookup address information: nodename nor servname provided, or not known\\n    2: failed to lookup address information: nodename nor servname provided, or not known')")


I need to find out John's name. The text explicitly states "My name is John".
Action: Search
Action Input: "What's my name?"Music video by China Anne McClain, Thomas Doherty, Dylan Playfair performing What ' s My Name (From "Descendants 2"/Sing-Along).© 2020 Walt Disney Records. Practice saying, " What ' s your name ?" and " My name is ...." with this fun and catchy song! Download a free mp3 of this song here: Dream English song by Matt R... Nothin' stays the same But I'm still in the game What ' s my name ? (Ringo!) Spinning round and round You broke the speed of sound Tearin' up the freeway With the tumblin' of the dice And... Ooh na na, what ’ s my name Ooh na na, what ’ s my name . hello what is your name song for kids.

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Final Answer: My name is John.

> Finished chain.
Response: My name is John.


## 3. Plan-and-Execute Pattern

This pattern first creates a plan, then executes each step.

In [81]:
# Simple Plan-and-Execute implementation
def plan_and_execute(task: str, llm, tools):
    """Simple plan-and-execute pattern"""
    
    # Step 1: Create a plan
    planning_prompt = f"""Break down this task into simple steps:
    Task: {task}
    
    Provide a numbered list of steps."""
    
    plan_response = llm.invoke(planning_prompt)
    plan = plan_response.content
    print(f"Plan:\n{plan}\n")
    
    # Step 2: Execute with agent
    agent = create_react_agent(llm, tools, react_prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    
    execution_prompt = f"""Execute this plan step by step:
    {plan}
    
    Original task: {task}"""
    
    result = executor.invoke({"input": execution_prompt})
    return result['output']

# Test plan-and-execute
complex_task = "Research the current price of Bitcoin, calculate what 10 Bitcoin would cost, and find recent news about cryptocurrency regulation"
result = plan_and_execute(complex_task, llm, tools)
print(f"Plan-and-Execute Result: {result}")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Plan:
Okay, here’s a breakdown of the task into simple, numbered steps:

1. **Find the Current Bitcoin Price:**
   * Use a reputable cryptocurrency exchange or financial website to find the current price of Bitcoin (BTC) in your preferred currency (e.g., USD, EUR, GBP).  Good sources include:
      * CoinMarketCap: [https://coinmarketcap.com/](https://coinmarketcap.com/)
      * Coinbase: [https://www.coinbase.com/](https://www.coinbase.com/)
      * Binance: [https://www.binance.com/](https://www.binance.com/)
      * Google Finance (search "Bitcoin price")

2. **Calculate the Cost of 10 Bitcoin:**
   * Multiply the current Bitcoin price by 10.  For example, if Bitcoin is trading at $65,000, then 10 Bitcoin would cost $650,000.

3. **Search for Recent Cryptocurrency Regulation News:**
   * Use a search engine (Google, Bing, etc.) to search for “recent cryptocurrency regulation news”.
   * Focus on news from reputable sources like:
      * Reuters: [https://www.reuters.com/](https://ww

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:ddgs.ddgs:Error in engine wikipedia: DDGSException("RuntimeError: RuntimeError('error sending request for url (https://wt.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=current%20bitcoin%20price): client error (Connect)\\n\\nCaused by:\\n    0: client error (Connect)\\n    1: dns error: failed to lookup address information: nodename nor servname provided, or not known\\n    2: failed to lookup address information: nodename nor servname provided, or not known')")


Okay, let's execute this plan step by step. I will start by finding the current Bitcoin price.
Action: Search
Action Input: "current bitcoin price"Oct 24, 2024 — The live Bitcoin price today is $ 112324.69 USD with a 24-hour trading volume of $48471501125.52 USD. We update our BTC to USD price in real-time. Aug 4, 2025 — Price of BTC is $ 112,092.16 right now . Looking at Bitcoin's historical prices, it's 3% up from the previous week's price of $108,864.82, and observed a 1% ... Jan 24, 2025 — The price of Bitcoin has decreased today. 1 BTC currently costs £83,247.95 , which represents a change of +0.53% in the last hour and a change of -0.07% in the ... 2 hours ago — Bitcoin has a current supply of 19,918,781. The last known price of Bitcoin is 111,646.55091185 USD and is down -0.31 over the last 24 hours. It is currently ... 4 days ago — Data is currently not available. Key Data. Daily Session High. 113260. Daily Session Low. 111121. Last Value . 112713. Net Change. 657.8. Percent Ch

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Okay, I have gathered the current Bitcoin price. It's $112,713. I will now proceed to calculate the cost of 10 Bitcoin and search for recent cryptocurrency regulation news.
Action: Calculator
Action Input: 112713 * 101127130

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:ddgs.ddgs:Error in engine wikipedia: DDGSException("RuntimeError: RuntimeError('error sending request for url (https://wt.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=recent%20cryptocurrency%20regulation%20news): client error (Connect)\\n\\nCaused by:\\n    0: client error (Connect)\\n    1: dns error: failed to lookup address information: nodename nor servname provided, or not known\\n    2: failed to lookup address information: nodename nor servname provided, or not known')")


Thought: Now that I have the current Bitcoin price ($112,713) and the cost of 10 Bitcoin ($1,127,130), I will proceed to search for recent cryptocurrency regulation news.
Action: Search
Action Input: "recent cryptocurrency regulation news"5 days ago · Crypto regulation has advanced: The Fed set an Oct. 21 stablecoin focus. The SEC and CFTC have allowed spot trading on registered exchanges. 5 days ago · Keep up with the latest crypto regulation . Dive into the latest news and updates to stay informed, maximize opportunities, and drive success in the crypto world. Aug 28, 2025 · Crypto Regulation news and analysis. Be Informed of the latest trends and events around regulator bodies and crypto assets. Aug 20, 2025 · From updates on regulatory frameworks in different countries to analysis of how government policies affect the crypto market, readers can stay informed on all aspects of the regulatory landscape. 5 days ago · Lawmakers in the Senate Banking Committee have a new draft of the cr

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Final Answer: The current Bitcoin price is $112,713. The cost of 10 Bitcoin is $1,127,130. Recent cryptocurrency regulation news includes updates on the Fed's stablecoin focus, the SEC and CFTC allowing spot trading on registered exchanges, and a new draft of the crypto market structure bill being considered by lawmakers in the Senate Banking Committee.

> Finished chain.
Plan-and-Execute Result: The current Bitcoin price is $112,713. The cost of 10 Bitcoin is $1,127,130. Recent cryptocurrency regulation news includes updates on the Fed's stablecoin focus, the SEC and CFTC allowing spot trading on registered exchanges, and a new draft of the crypto market structure bill being considered by lawmakers in the Senate Banking Committee.


## 4. Multi-Agent Orchestration

Create specialized agents for different tasks.

In [82]:
# Create specialized agents
research_tools = [Tool(name="Search", func=search.run, description="Research information")]
math_tools = [Tool(name="Calculator", func=calculator, description="Perform calculations")]

research_agent = create_react_agent(llm, research_tools, react_prompt)
math_agent = create_react_agent(llm, math_tools, react_prompt)

research_executor = AgentExecutor(agent=research_agent, tools=research_tools)
math_executor = AgentExecutor(agent=math_agent, tools=math_tools)

# Orchestrate agents
def orchestrate_agents(task: str):
    # Step 1: Research
    research_result = research_executor.invoke({"input": f"Research: {task}"})
    
    # Step 2: Extract numbers and calculate
    math_task = f"Based on this research: {research_result['output']}, perform any necessary calculations"
    math_result = math_executor.invoke({"input": math_task})
    
    return f"Research: {research_result['output']}\n\nCalculations: {math_result['output']}"

multi_agent_task = "Find the current population of Tokyo and calculate how many people that is per square kilometer if Tokyo is 2194 km²"
result = orchestrate_agents(multi_agent_task)
print(f"Multi-Agent Result:\n{result}")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:ddgs.ddgs:Error in engine wikipedia: DDGSException("RuntimeError: RuntimeError('error sending request for url (https://wt.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=current%20population%20of%20Tokyo): client error (Connect)\\n\\nCaused by:\\n    0: client error (Connect)\\n    1: dns error: failed to lookup address information: nodename nor servname provided, or not known\\n    2: failed to lookup address information: nodename nor servname provided, or not known')")
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:ddgs.ddgs:Error in engine wikipedia: DDGSException("RuntimeError: RuntimeError('error sending request for url (https://wt.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=Tokyo%20population%20area): client error (Connect)\\n\\nCaused by:\\n    0: client error (Connect)\\n    1: dns error: failed to lookup ad

Multi-Agent Result:
Research: 8,300,000 / 2194 = 380.26 people per square kilometer.

Calculations: 3783.0446672743847


## 5. Advanced Tool Integration with State

In [83]:
# Stateful tool example
class TaskTracker:
    def __init__(self):
        self.tasks = []
    
    def add_task(self, task: str) -> str:
        self.tasks.append(task)
        return f"Added task: {task}. Total tasks: {len(self.tasks)}"
    
    def list_tasks(self, _: str = "") -> str:  # Accept dummy argument
        if not self.tasks:
            return "No tasks found"
        return "Tasks: " + ", ".join(self.tasks)

tracker = TaskTracker()

stateful_tools = [
    Tool(name="AddTask", func=tracker.add_task, description="Add a task to track"),
    Tool(name="ListTasks", func=tracker.list_tasks, description="List all tasks")
]

stateful_agent = create_react_agent(llm, stateful_tools, react_prompt)
stateful_executor = AgentExecutor(agent=stateful_agent, tools=stateful_tools)

# Test stateful operations
print("Adding tasks:")
result1 = stateful_executor.invoke({"input": "Add task: Review code"})
result2 = stateful_executor.invoke({"input": "Add task: Write tests"})
result3 = stateful_executor.invoke({"input": "List all my tasks"})
print(f"Final result: {result3['output']}")

Adding tasks:


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Final result: Tasks: Review code, Write tests


## 6. RAG-Enabled Agent

Building on the local_rag.ipynb concepts with Ollama embeddings.

In [84]:
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader

# Load and process documents
loader = DirectoryLoader('./data/markdowns/', glob="*.md")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(texts, embeddings, persist_directory="./chroma_agents")

# Create RAG tool
def rag_search(query: str) -> str:
    """Search through local documents"""
    docs = vectorstore.similarity_search(query, k=3)
    return "\n\n".join([doc.page_content for doc in docs])

rag_tools = [
    Tool(name="DocumentSearch", func=rag_search, description="Search through local documents"),
    Tool(name="Calculator", func=calculator, description="Calculate math expressions")
]

rag_agent = create_react_agent(llm, rag_tools, react_prompt)
rag_executor = AgentExecutor(agent=rag_agent, tools=rag_tools, verbose=True)

# Test RAG agent
rag_query = "What Linux commands are mentioned in the documents and how would you calculate their total count?"
result = rag_executor.invoke({"input": rag_query})
print(f"RAG Agent Result: {result['output']}")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"




> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


I need to search the documents for Linux commands and then calculate the total count.
Action: DocumentSearch
Action Input: "Linux commands"class SERVER_COMMANDS:
    UPLOAD_CMD = 0x0001
    QUERY_CMD = 0x0002

Since we only have two types of data we are transmitting to the server, we only need to define two commands here, but we could easily expand the protocol by defining more later. Each is given a numeric value, which will be the 2-byte value that is actually transmitted over the network to determine the use case. We could define a larger size to allow for larger numbers of commands, or use a 1-byte value (or even just one single bit that was combined with other data like the size of the transmission) to minimize bandwidth usage, since we only have two commands, but this is a good starting point.

class SERVER_COMMANDS:
    UPLOAD_CMD = 0x0001
    QUERY_CMD = 0x0002

Since we only have two types of data we are transmitting to the server, we only need to define two commands here, but

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Final Answer: The documents mention two Linux commands: UPLOAD_CMD and QUERY_CMD. The total count is 2.


> Finished chain.
RAG Agent Result: The documents mention two Linux commands: UPLOAD_CMD and QUERY_CMD. The total count is 2.


## 7. Production Considerations

When deploying agents to production, you need proper error handling, logging, and monitoring. Key considerations include:

- **Error Handling**: Graceful failure when tools fail or parsing errors occur
- **Logging**: Track agent actions, decisions, and performance
- **Timeouts**: Prevent infinite loops or long-running operations
- **Rate Limiting**: Control API usage and costs
- **Monitoring**: Track success rates, response times, and errors

In [85]:
# Production configuration with error handling and logging
from langchain_core.callbacks import BaseCallbackHandler
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ProductionCallbackHandler(BaseCallbackHandler):
    def on_agent_action(self, action, **kwargs):
        logger.info(f"Agent Action: {action.tool} - {action.tool_input}")
    
    def on_agent_finish(self, finish, **kwargs):
        logger.info(f"Agent Finished: {finish.return_values}")
    
    def on_chain_error(self, error, **kwargs):
        logger.error(f"Agent Error: {error}")

production_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    callbacks=[ProductionCallbackHandler()],
    max_iterations=10,
    early_stopping_method="generate",
    handle_parsing_errors=True
)

# Test production setup
result = production_executor.invoke({"input": "What is 2+2?"})
print(f"Production result: {result['output']}")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:__main__:Agent Action: Calculator - 2+2
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:__main__:Agent Finished: {'output': '4'}


Production result: 4


## 8. Custom Agent Pattern: Workflow Agent

In [86]:
# Custom workflow agent that manages multi-step processes
class WorkflowAgent:
    def __init__(self, llm, tools):
        self.llm = llm
        self.tools = {tool.name: tool for tool in tools}
        self.workflow_state = {}
    
    def execute_workflow(self, steps: list) -> dict:
        results = {}
        
        for i, step in enumerate(steps):
            step_name = f"step_{i+1}"
            print(f"Executing {step_name}: {step['description']}")
            
            if step['tool'] in self.tools:
                tool = self.tools[step['tool']]
                result = tool.func(step['input'])
                results[step_name] = {
                    'description': step['description'],
                    'result': result
                }
                # Store result for next steps
                self.workflow_state[step_name] = result
            else:
                results[step_name] = {'error': f"Tool {step['tool']} not found"}
        
        return results

# Create workflow agent
workflow_agent = WorkflowAgent(llm, tools)

# Define a workflow
workflow_steps = [
    {
        'description': 'Search for current Bitcoin price',
        'tool': 'Search',
        'input': 'current Bitcoin price USD'
    },
    {
        'description': 'Calculate cost of 5 Bitcoin',
        'tool': 'Calculator', 
        'input': '50000 * 5'  # Assuming $50k per Bitcoin
    }
]

# Execute workflow
workflow_results = workflow_agent.execute_workflow(workflow_steps)
print("\nWorkflow Results:")
for step, result in workflow_results.items():
    print(f"{step}: {result}")

INFO:ddgs.ddgs:Error in engine wikipedia: DDGSException("RuntimeError: RuntimeError('error sending request for url (https://wt.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=current%20Bitcoin%20price%20USD): client error (Connect)\\n\\nCaused by:\\n    0: client error (Connect)\\n    1: dns error: failed to lookup address information: nodename nor servname provided, or not known\\n    2: failed to lookup address information: nodename nor servname provided, or not known')")


Executing step_1: Search for current Bitcoin price
Executing step_2: Calculate cost of 5 Bitcoin

Workflow Results:
step_1: {'description': 'Search for current Bitcoin price', 'result': 'We update our BTC to USD price in real-time.The current valuation of Bitcoin is constantly moving, all day every day. It is a truly global asset. From a start of under one cent per coin, BTC has risen in price by thousands of percent to the numbers you see above. Aug 4, 2025 — Get the latest price, news, live charts, and market trends about Bitcoin. The current price of Bitcoin in United States is $ 112092.16 per (BTC / USD ) 2 hours ago — US Dollar, Vietnamese Dong. Convert. Conversion Value. 0.854409 . Top Stories On CoinDesk Bitcoin Price Index (XBX). Read full story. Cryptocurrencies Bitcoin, ... Aug 20, 2025 — BTCUSD Bitcoin US DollarCurrency Exchange Rate Live Price Chart. Chart; Quotes; Financials; Alerts. Exchange Rate. 112298 . Daily Change. 767.0000 0.69%. Monthly. 5 hours ago — Bitcoin fell 

## 9. Framework Comparison: LangChain vs AWS Strands

While this notebook focuses on LangChain, let's compare it with AWS Strands to understand the trade-offs between comprehensive ecosystems and minimal-code approaches.

In [87]:
# AWS Strands with Ollama - Minimal approach
# !pip install strands-agents

from strands import Agent
from strands.models.ollama import OllamaModel

# Create an Ollama-backed model
ollama_model = OllamaModel(
    host="http://localhost:11434",  # Ollama server URL
    model_id="gemma3"               # Specify the model you pulled
)

# Build your agent (minimal code)
strands_agent = Agent(model=ollama_model)

# Use the agent
response = strands_agent("What is 25 * 4?")
print(f"Strands Response: {response}")

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


25 * 4 = 100
Strands Response: 25 * 4 = 100




In [88]:
# Minimal LangChain with Ollama (comparable to Strands)
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(model="gemma3", temperature=0)
prompt = ChatPromptTemplate.from_template("Answer briefly: {question}")
chain = prompt | llm | StrOutputParser()

response = chain.invoke({"question": "What is 25 * 4?"})
print(f"LangChain Response: {response}")


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


LangChain Response: 100



### **LangChain vs AWS Strands Comparison**

| Aspect | LangChain | AWS Strands |
|--------|-----------|-------------|
| **Code Complexity** | Medium (flexible; can be concise) | Low (minimal code) |
| **Lines of Code** | 6–20 depending on pattern | 3–8 for simple agent |
| **Learning Curve** | Moderate–steep (many concepts) | Gentle (opinionated) |
| **Customization** | Highly flexible | Opinionated patterns |
| **Ecosystem** | Extensive (1000+ integrations) | AWS-focused |
| **Memory Management** | Pluggable memory components | Opinionated defaults |
| **Tool Integration** | Manual or schema-based; rich ecosystem | Simplified wiring; AWS-first |
| **Multi-Agent** | LangGraph patterns built-in | Built-in patterns |
| **Error Handling** | Configurable; retries/callbacks available | Built-in |
| **Production Ready** | Portable; requires configuration | AWS-native deployment |
| **Local Models** | Full support (Ollama, etc.) | Ollama support |
| **Cloud Integration** | Multi-cloud | AWS-optimized |
| **Observability** | LangSmith/tracing callbacks | CloudWatch/X-Ray integration |
| **Guardrails/Evals** | Prompt/versioning, policy hooks, eval libs | AWS guardrails/services |
| **Governance & Security** | BYO controls per platform | IAM/VPC/Data residency via AWS |
| **Vendor Lock-in** | Low (portable) | Higher (AWS services) |
| **Community** | Large, active | Growing |
| **Documentation** | Extensive | Focused |
| **Debugging** | Verbose/loggable | Abstracted |
| **Performance** | Depends on configuration | Optimized defaults |

### **When to Choose Each Framework**

**Choose LangChain when:**
- You need maximum flexibility and customization
- Working with multiple LLM providers or cloud platforms
- Building complex, custom agent architectures
- Learning agent concepts in depth
- Have existing LangChain infrastructure
- Need extensive tool ecosystem

**Choose AWS Strands when:**
- You want rapid prototyping and deployment
- Building primarily on AWS infrastructure
- Need production-ready agents quickly
- Prefer opinionated, best-practice patterns
- Want minimal code maintenance
- Trust AWS defaults and optimizations

### **Code Comparison Summary**

**Strands (3 lines):**
```python
ollama_model = OllamaModel(host="http://localhost:11434", model_id="gemma3")
agent = Agent(model=ollama_model)
response = agent("What is 25 * 4?")
```

**LangChain (6 lines, minimal):**
```python
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(model="gemma3", temperature=0)
prompt = ChatPromptTemplate.from_template("Answer briefly: {question}")
response = (prompt | llm | StrOutputParser()).invoke({"question": "What is 25 * 4?"})
```

Both approaches have merit - LangChain for learning and flexibility, Strands for speed and simplicity.

## Summary: Agent Architecture Patterns

This notebook demonstrated key agent patterns using LangChain with local Ollama models:

###  **Patterns Covered:**
1. **ReAct Pattern** - Reasoning and Acting in loops
2. **Plan-and-Execute** - Strategic planning before execution
3. **Multi-Agent** - Specialized agents working together
4. **Workflow Agent** - Custom orchestration patterns

### **Core Capabilities:**
- **Tool Integration** - Calculator, search, document retrieval
- **Memory Management** - Conversation history and state
- **State Persistence** - Stateful tools and workflows
- **Error Handling** - Production-ready error management